In [ ]:
def simplex(matrix, rhs, z, varlabel='x', direction=1):
    '''Simplex algorithm to solve linear programming problems

    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side

    rhs: numpy ndarray
        Right-hand side vector


    direction: {+1 , -1}
        Use +1 for maximization and -1 for minimization.

    Returns
    -------
    solutions: Pandas DataFrame
        Vector solutions of every iteration

    lastrows: pandas DataFrame
        Last rows of optimal table, this rows corresponde to Zj and cj - Zj
    
    optimal_table: Pandas DataFrame
        Optimal Table
    
    b: pandas Series
        Vector solution
    
    cbindex: numpy ndarray
        Index of coefficient basics in z vector
    '''
    print("="*10, "Simplex Method", "="*10)

    matrix = np.array(matrix, dtype=float)
    rhs = np.array(rhs, dtype=float)
    z = np.array(z, dtype=float)

    num_rows, num_cols = matrix.shape

    cb_index = np.where((matrix == 1) & (matrix.sum(axis=0) == 1))[1]
    
    cb = z[cb_index]

    zj = cb.dot(matrix)

    net_evaluation = direction * (z - zj)

    solutions = []
    

    labels = [f"{varlabel}{i + 1}" for i in range(num_cols)]
    basis = [labels[i] for i in cb_index]
    
    
    iteration = 0
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)
        entering_label = labels[entering]

        key_col = matrix[:, entering]

        ratios = np.divide(rhs, key_col,
                           out=np.full_like(rhs, np.inf),
                           where=key_col > 0)

        leaving = ratios.argmin()   # leaving variables (index)
        leaving_label = labels[cb_index[leaving]]
        # ==================
        pivot = matrix[leaving, entering]

        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot

        for i in range(num_rows):
            if i != leaving:
                factor = matrix[i, entering]
                matrix[i] = -factor * matrix[leaving] + matrix[i]
                rhs[i] = -factor * rhs[leaving] + rhs[i]
        # ==================
        cb_index[leaving] = entering

        cb = z[cb_index]
        zj = cb.dot(matrix)
        net_evaluation = direction * (z - zj)

        solution[cb_index] = rhs  # basics

        iteration += 1
        
        basis[leaving] = entering_label  

        solutions.append(solution)
        
        if np.all(net_evaluation <= 0):
            print(f"Optimal solution found in {iteration} iterations")
            print(f'Basis: {basis}')
            print(f'Cb: {cb}')
            print(f'Optimal Table: \n {matrix} \n')
            print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}")
            
            

    return pd.DataFrame(np.array(solutions), index=range(1, iteration + 1), columns=labels),\
pd.DataFrame( np.vstack((zj, net_evaluation)), index=['zj', 'cj - zj'], columns=labels),\
pd.DataFrame(matrix, index=basis, columns=labels),\
pd.Series(rhs, index=basis, name='solution'),\
cb_index